# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298234454068                   -0.85    5.5
  2   -8.300180574648       -2.71       -1.25    1.0
  3   -8.300445901030       -3.58       -1.89    3.4
  4   -8.300462335639       -4.78       -2.76    1.5
  5   -8.300464465472       -5.67       -3.05    9.8
  6   -8.300464544260       -7.10       -3.21    1.1
  7   -8.300464593210       -7.31       -3.35    2.4
  8   -8.300464619446       -7.58       -3.49    1.0
  9   -8.300464636827       -7.76       -3.67    1.0
 10   -8.300464640449       -8.44       -3.79    1.0
 11   -8.300464643353       -8.54       -4.04    1.1


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67437321981                   -0.70    7.2
  2   -16.67871546789       -2.36       -1.14    1.0
  3   -16.67923344178       -3.29       -1.87    3.2
  4   -16.67927842594       -4.35       -2.78    3.4
  5   -16.67928607476       -5.12       -3.21    4.4
  6   -16.67928621673       -6.85       -3.50    4.1
  7   -16.67928622262       -8.23       -3.97    1.1
  8   -16.67928622433       -8.77       -4.62    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.31907804638                   -0.56    6.9
  2   -33.32613948527       -2.15       -1.00    1.8
  3   -33.33600436676       -2.01       -1.67    4.4
  4   -33.33616418504       -3.80       -2.56    3.6
  5   -33.33688996731       -3.14       -2.46   11.1
  6   -33.33691822791       -4.55       -2.62    3.2
  7   -33.33694354087       -4.60       -3.42    3.4
  8   -33.33694391920       -6.42       -3.87    3.9
  9   -33.33694392663       -8.13       -4.71    1.1


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298718577928                   -0.85    5.0
  2   -8.300292999827       -2.80       -1.59    1.0
  3   -8.300421531404       -3.89       -2.39    4.8
  4   -8.300318126891   +   -3.99       -2.17   12.9
  5   -8.300464557448       -3.83       -3.80    2.5
  6   -8.300464618569       -7.21       -4.10    3.2


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.29741900996                   -0.56    6.9
  2   -33.24930073681   +   -1.32       -1.24    1.4
  3   +15.29531890614   +    1.69       -0.22   11.6
  4   -33.30609078453        1.69       -1.50    6.9
  5   -33.24472411261   +   -1.21       -1.35    4.2
  6   -31.87253770747   +    0.14       -0.98    4.1
  7   -33.26236372434        0.14       -1.61    4.8
  8   -33.33564614804       -1.14       -2.12    3.5
  9   -33.33571204432       -4.18       -2.19    3.4
 10   -33.33654659662       -3.08       -2.49    2.2
 11   -33.33692019125       -3.43       -2.93    2.0
 12   -33.33692052314       -6.48       -2.92    2.0
 13   -33.33692278580       -5.65       -2.92    1.5
 14   -33.33692291773       -6.88       -2.91    1.5
 15   -33.33691949594   +   -5.47       -2.84    1.8
 16   -33.33692433850       -5.31       -2.90    1.0
 17   -33.33692816326       -5.42       -3.04 

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.